In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate

In [6]:
# resource = 'https://drive.google.com/uc?export=download&id=1by5_2HhJ4WO5VBJ0Tc9OZStPqDwiAUYW'
resource = 'https://drive.google.com/uc?export=download&id=15tURO14aosIqB2pIc-fJQ_pgHAPM8Ho-'
dataset = pd.read_csv(resource)

## Definir Ponderacion
___

- Ponderaciones asignadas al programa académico de Ingeniería de Sistemas (UP)
  
|Ciencias Naturales|Lectura Crítica|Matemáticas|Sociales y Ciudadanias|Inglés|
|--|--|--|--|--|
|45%|15%|35%|0%|5%| 

## Proceso de Selección
___

- Multiplicacion del ponderado asignado a cada una de las áreas del ICFES 
   de la universidad por el valor de los resultados del Exámen ICFES Saber 11


In [7]:
def selection_process(dataset):
    # Define ponderation
    nature_sciences = 0.45
    critical_reading = 0.15
    maths = 0.35
    sociology = 0
    english = 0.05

    # create sub_dataset 
    columns=['punt_lectura_critica',
            'punt_matematicas',
            'punt_c_naturales',
            'punt_sociales_ciudadanas',
            'punt_ingles',
            'punt_global']

    df = pd.DataFrame(data=dataset,index=None,columns=columns)

    # transform_columns

    df['punt_lectura_critica'] = df['punt_lectura_critica'] * critical_reading
    df['punt_matematicas'] = df['punt_matematicas'] * maths
    df['punt_c_naturales'] = df['punt_c_naturales'] * nature_sciences
    df['punt_sociales_ciudadanas'] = df['punt_sociales_ciudadanas'] * sociology
    df['punt_ingles'] = df['punt_ingles'] * english

    # calculate punt_ponderate
    df['punt_ponderado'] = df.iloc[:,:5].sum(axis=1)

    # move the last column to penult position
    df_cols = df.columns.tolist()
    reorder = columns[:-1] + df_cols[-1:]
    reorder.append(columns[-1])
    reorder

    df = df[reorder]
    return df.sort_values(by=['punt_ponderado','punt_global'], ascending=False)

### Porcentaje de Selección
___
- Para seleccionar un top de 5 puestos, se define:

|Puesto|%Ponderado|
|--|--|
|1|> 60%|
|2|[55% - 60%)|
|3|[45% - 55%)|
|4|[35% - 45%)|
|5|< 35%|


#

In [21]:
# conditions
def define_top(df):
    df_positions = df.copy()
    df_positions['puesto'] = np.where(df_positions.punt_ponderado>60,1,
                                np.where((df_positions.punt_ponderado>=55) & (df_positions.punt_ponderado<60),2,
                                np.where((df_positions.punt_ponderado>=45) & (df_positions.punt_ponderado<55),3,
                                np.where((df_positions.punt_ponderado>=35) & (df_positions.punt_ponderado<45),4,
                                np.where(df_positions.punt_ponderado<35,5,0)))))
    return df_positions.sort_values(by=['punt_ponderado','punt_global'], ascending=False)

df = selection_process(dataset)
df_positions= define_top(df)

df_positions.shape


(15435, 8)

### Estadísticas de Admisión UP
___
- Aplica solo para programa Ing de Sistemas
  - **Notas Aclaratorias**
    - El puntaje corresponde a los obtenidos por los aspirantes en la línea de corte desde el proceso de admisión de __PrimerSemestre 2017__ y __SegundoSemestre 2021__. Estos puntajes de corte no representan los requisitos de inscripción y/o admisión.
    - La información hace referencia a datos estadísticos. Los puntajes de corte para el próximo proceso de admisión pueden variar y depende únicamente de los puntajes de los aspirantes inscritos en cada proceso de admisión.
     


## Regresión Logística
- En estadística, la regresión logística es un tipo de análisis de regresión utilizado para predecir el resultado de una variable categórica (una variable que puede adoptar un número limitado de categorías) en función de las variables independientes o predictoras. Es útil para modelar la probabilidad de un evento ocurriendo en función de otros factores. El análisis de regresión logística se enmarca en el conjunto de Modelos Lineales Generalizados (GLM por sus siglas en inglés) que usa como función de enlace la función logit. En este modelo, las probabilidades que describen los posibles resultados de un solo ensayo se modelan utilizando una función logística.

## Objetivo

- El objetivo es predecir si un aspirante de ICFES Saber 11, puede acceder al programa de Ing de Sistemas en una institución de calidad superior como lo es la Universidad de Pamplona.

### Lectura de Dataset
___

- El dataset histórico, está alimentado por 88 datos, que serán tomados en cuenta 
  para la regresión. 

In [9]:
url = 'https://drive.google.com/uc?export=download&id=1Ld0oPw01AWrJE7xj3M-8szK0ORq6Lj0P'
historic_up = pd.read_csv(url)

In [10]:
# Dimensión Dataset
historic_up.shape

(88, 1)

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [15]:
# feature entineering (one hot encoding)
# saber_11 = pd.get_dummies(df_positions['punt_ponderado'])
# saber_11

# Variables de Predicción
X = historic_up.Punt_obtenido_2017_1 
Y = historic_up.Punt_obtenido_2017_1 # target
# Segment data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.7)
scalar = StandardScaler()

